In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from sklearn.model_selection import train_test_split
# from category_encoders import TargetEncoder

# Preparing for modeling
From the analyzed and preprocessed data, we continue the pre-processing in order to prepare it for the modeling phase

In [4]:
df = pd.read_csv('../data/data2.csv')


In [59]:
print( df.keys() )
df = df.head()
df[["opponent", "shot_made_flag"]]

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'period', 'playoffs', 'season',
       'shot_distance', 'shot_made_flag', 'shot_type', 'shot_zone_area',
       'shot_zone_basic', 'shot_zone_range', 'team_id', 'team_name',
       'game_date', 'matchup', 'opponent', 'shot_id', 'time_remaining',
       'last_5_games_avg', 'streak_before_shot', 'points_before_shot',
       'fgp_before_shot', 'month'],
      dtype='object')


,opponent,shot_made_flag
0,MIN,0.0
1,NYK,0.0
2,CHA,1.0
3,CHA,0.0
4,CHA,1.0


#### The encoding looks like

In [64]:
encoding = df.groupby("opponent")["shot_made_flag"].mean().reset_index()
encoding = encoding.rename(columns={"shot_made_flag": "opponent_te"})
encoding.head()

,opponent,opponent_te
0,ATL,0.452055
1,BKN,0.400000
2,BOS,0.411239
3,CHA,0.436000
4,CHI,0.430233


In [65]:
target_encoding(df, "opponent", "shot_made_flag")

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,matchup,opponent,shot_id,time_remaining,last_5_games_avg,streak_before_shot,points_before_shot,fgp_before_shot,month,opponent_te
0,Jump Shot,Jump Shot,102,29600027,33.9283,-140,116,-118.4098,1,0,...,home,MIN,22902,42,0.00,0,0,0.00,11,0.444627
1,Jump Shot,Jump Shot,127,29600031,33.9473,-131,97,-118.4008,2,0,...,away,NYK,22903,608,0.00,0,0,0.00,11,0.477032
2,Jump Shot,Jump Shot,124,29600044,33.8633,-142,181,-118.4118,2,0,...,away,CHA,22904,517,0.00,0,0,0.00,11,0.436000
3,Jump Shot,Jump Shot,144,29600044,34.0443,0,0,-118.2698,2,0,...,away,CHA,22905,394,0.00,1,3,1.00,11,0.436000
4,Jump Shot,Jump Shot,151,29600044,33.9063,-10,138,-118.2798,2,0,...,away,CHA,22906,327,0.00,0,3,0.50,11,0.436000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692,Layup Shot,Layup,230,21501228,34.0453,-19,-1,-118.2888,2,0,...,home,UTA,22870,84,0.29,0,16,0.41,4,0.444265
25693,Fadeaway Jump Shot,Jump Shot,251,21501228,33.9333,40,111,-118.2298,2,0,...,home,UTA,22871,0,0.29,0,16,0.39,4,0.444265
25694,Driving Layup Shot,Layup,266,21501228,34.0373,-2,7,-118.2718,3,0,...,home,UTA,22872,633,0.29,0,16,0.37,4,0.444265
25695,Jump Shot,Jump Shot,281,21501228,33.7313,-32,313,-118.3018,3,0,...,home,UTA,22876,540,0.29,0,20,0.41,4,0.444265


In [5]:
def target_encoding(df, col_to_encode, target_col):
    """
    Performs target-encoding on the specified column. That is to convert categorical column 
    to a numerical column.

    Args:
        df (DataFrame):
        col_to_encode (str): Column to be converted to numerical one.
        target_col (str): Target column of the data.

    Returns:
        DataFrame: DataFrame containing the new <col_to_encode>_te column.
    """

    encoding = df.groupby(col_to_encode)[target_col].mean().reset_index()
    encoding = encoding.rename(columns={target_col: f"{col_to_encode}_te"})
    df = df.merge(right=encoding, how='left', on=col_to_encode)
    return df

In [ ]:
encodings = data.groupby('Country')['Target Variable'].mean().reset_index()

data = data.merge(encodings, how='left', on='Country')

data.drop('Country', axis=1, inplace=True)

In [10]:
# def convert_matchup(df):
#     df['isHome'] = df.matchup.apply(lambda x: 1 if  x=="home" else 0)
#     df = df.drop(['matchup'], axis=1)
#     return df
# df = convert_matchup(df)

In [11]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return res

In [12]:
def one_hot_encoding(df):
    df_new = df.copy(deep=True)
    features_to_encode = ['combined_shot_type', 'season', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'opponent', 'action_type', 'matchup']
    for feature in features_to_encode:
        df_new = encode_and_bind(df_new, feature)

    return df_new

In [13]:
def target_encoding(df_x, df_y):
    encoder = TargetEncoder()
    features_to_encode = ['season', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'opponent', 'action_type', 'matchup']
    # print(df_x.columns)
    # print(type(df_y))
    # for feature in features_to_encode:
    df_x[f'encoded {feature}'] = encoder.fit_transform(df_x, df_y)
    df_x.drop([feature], axis=1, inplace=True)
    return df_x, df_y

    # return df

In [17]:
x_train, y_train, x_test, y_test = split_team_season(df, 'POR', ['2000-01'], ['2002-03', '2003-04'] )

NameError: name 'split_team_season' is not defined

In [79]:
x,y = split_x_y(df, 'Yearly Salary in Thousands')

In [82]:
# error when in for
encoder = TargetEncoder()
for column in ['team_name', 'opponent']:
    print("in")
    x_train[f'encoded_{column}'] = encoder.fit_transform(x_train[column], y_train)
    print("out")
    # x_train.drop(['season'], axis=1, inplace=True)

in
out
in


KeyError: 'team_name'

In [22]:
x_train

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,opponent,shot_id,time_remaining,points,season_avg,plus_minus,FGP,month,encoded,encoded_team_name
0,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,1,0,...,POR,2,622,8,18.26,-10.26,0.444,10,0.477064,0.477064
1,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,1,0,...,POR,3,465,8,18.26,-10.26,0.444,10,0.477064,0.477064
2,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,1,0,...,POR,4,412,8,18.26,-10.26,0.444,10,0.477064,0.477064
3,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,2,0,...,POR,5,379,8,18.26,-10.26,0.444,10,0.477064,0.477064
4,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,3,0,...,POR,6,572,8,18.26,-10.26,0.444,10,0.477064,0.477064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21978,Running Jump Shot,Jump Shot,214,40000022,34.0353,41,9,-118.2288,2,1,...,POR,26245,88,14,18.26,-4.26,0.389,4,0.477064,0.477064
21979,Jump Shot,Jump Shot,261,40000022,33.9953,-202,49,-118.4718,3,1,...,POR,26247,584,14,18.26,-4.26,0.389,4,0.477064,0.477064
21980,Jump Shot,Jump Shot,336,40000022,34.0403,-195,4,-118.4648,3,1,...,POR,26250,33,14,18.26,-4.26,0.389,4,0.477064,0.477064
21981,Turnaround Jump Shot,Jump Shot,385,40000022,33.9573,-166,87,-118.4358,4,1,...,POR,26251,457,14,18.26,-4.26,0.389,4,0.477064,0.477064


In [14]:
target_encoding(x_train, y_train)

NameError: name 'x_train' is not defined

# Splitting the dataset

In [158]:
x_test

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,opponent,shot_id,time_remaining,points,season_avg,plus_minus,FGP,month,encoded season,encoded shot_type
2650,Dunk Shot,Dunk,3,20200014,34.0443,0,0,-118.2698,1,0,...,POR,3137,710,17,18.49,-1.49,0.438,10,0.533333,0.557522
2651,Turnaround Jump Shot,Jump Shot,17,20200014,34.0443,174,0,-118.0958,1,0,...,POR,3139,574,17,18.49,-1.49,0.438,10,0.533333,0.557522
2652,Jump Shot,Jump Shot,65,20200014,33.8023,49,242,-118.2208,1,0,...,POR,3140,232,17,18.49,-1.49,0.438,10,0.533333,0.307692
2653,Alley Oop Layup shot,Layup,71,20200014,34.0443,0,0,-118.2698,1,0,...,POR,3142,178,17,18.49,-1.49,0.438,10,0.533333,0.557522
2654,Jump Shot,Jump Shot,115,20200014,33.8303,0,214,-118.2698,2,0,...,POR,3143,710,17,18.49,-1.49,0.438,10,0.533333,0.557522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5208,Driving Finger Roll Shot,Layup,525,20301187,34.0443,0,0,-118.2698,6,0,...,POR,6205,211,30,14.76,15.24,0.481,4,0.454545,0.557522
5209,Fadeaway Jump Shot,Jump Shot,532,20301187,33.9293,146,115,-118.1238,6,0,...,POR,6206,147,30,14.76,15.24,0.481,4,0.454545,0.557522
5210,Jump Shot,Jump Shot,538,20301187,33.7973,15,247,-118.2548,6,0,...,POR,6207,102,30,14.76,15.24,0.481,4,0.454545,0.307692
5211,Jump Shot,Jump Shot,546,20301187,33.8523,-133,192,-118.4028,6,0,...,POR,6208,9,30,14.76,15.24,0.481,4,0.454545,0.557522


## Random
### Like this?

In [54]:
def split_random(df, train=None, test=None, validation=None):
    # 80-20
    train_data, rest_data = train_test_split(df, train_size=0.8, shuffle=False)
    # test 10 val 10
    validation_data, test_data = train_test_split(rest_data, test_size=0.5, shuffle=False)

    print(f"Size of training dataset {len(train_data)}" )
    print(f"Size of validation dataset {len(validation_data)}" )
    print(f"Size of test dataset {len(test_data)}" )

    # print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )
    return train_data, validation_data, test_data

In [55]:
a,b,c = split_random(df)

Size of training dataset 20557
Size of validation dataset 2570
Size of test dataset 2570


## Regular (train), playoff (test)
### Do we need train, test, validation here?

In [8]:
def split_x_y(df, target):
    y = df.loc[:, [target]]
    x = df.drop([target], axis=1)
    return x, y

In [9]:
def split_regular_playoff(df, season, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_validaiton = None

    df_season = df[df['season'] == season]
    isPlayoffs = df_season.playoffs == 1

    df_train = df_season[~isPlayoffs]
    df_test  = df_season[isPlayoffs]

    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    train = split_x_y(df_train, 'shot_made_flag')
    test = split_x_y(df_test, 'shot_made_flag')
    return train[0], train[1], test[0], test[1]

In [10]:
x_train, y_train, x_test, y_test = split_regular_playoff(df, '2010-11')

Size of training dataset 1360
Size of test dataset 161
Size of train vs test ratio 0.8941485864562788


## Pick one team, train seasons before, test seasons after
### Should we train on multiple seasons?


In [11]:
def split_team_season(df, opponent,  season_train, season_test, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_train = None
    df_test = None
    df_validaiton = None

    if season_train == [] or season_test == []:
        print("Empty seasons")
        return None

    df_train = df[df['season'].isin(season_train)  & (df['opponent'] == opponent) ]
    df_test = df[(df['season'].isin(season_test)) & (df['opponent'] == opponent) ]
    print(f"Testing vs {opponent}\ntraining: {season_train}\ntesting: {season_test}" )
    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    train = split_x_y(df_train, 'shot_made_flag')
    test = split_x_y(df_test, 'shot_made_flag')
    return train[0], train[1], test[0], test[1]

In [61]:
x_train, y_train, x_test, y_test = split_team_season(df, 'POR', ['2000-01'], ['2002-03', '2003-04'] )


Testing vs POR
training: ['2000-01']
testing: ['2002-03', '2003-04']
Size of training dataset 109
Size of test dataset 152
Size of train vs test ratio 0.41762452107279696


KeyError: "'season'"

In [ ]:
# df.to_csv('../data/data3.csv', index=False)